In [45]:
# Source: https://github.com/prestinomills/aqueduct/blob/Know_Your_Community_Pipelines/civis/geohub/ActiveBusinessBlockgroupAggregation/Active_Business_Finalized_Script.py
# %load active_business_script.py
"""
Created on Wed May  1 08:51:03 2019
@author: myrfid041
"""
import geopandas as gpd
import os
import pandas as pd

import esri_credentials

from sodapy import Socrata
from arcgis.gis import GIS
from arcgis.features.summarize_data import join_features
from IPython.display import display
from arcgis.features import FeatureLayer, FeatureLayerCollection

lahub_user = os.environ["LAHUB_ACC_USERNAME"]
lahub_pass = os.environ["LAHUB_ACC_PASSWORD"]


#---Setting the Outputs
OUTPUT_FILE = "./Listing_of_Active_Businesses.csv"
output_layer_name = '067a9242fbef4afeb1ca0744952e5724'
max_record_count = 250_000

#---Pulling Active Business Data
client = Socrata("data.lacity.org", None)
#abiz = pd.DataFrame(client.get('ngkp-kqkn', limit=10000000))
abiz = pd.read_pickle("../data/abiz.p")


#---Pull NAIC Industry Table
n_table=(
    'https://raw.githubusercontent.com/CityofLosAngeles/civis-gcp-transition/{}/'
    'data/naics_industry_table.csv'
)
naics_table=pd.read_csv(n_table.format("active-business"))



def dataprep(df,naics_table):
    # Grab location info
    df = (df.dropna(subset=['location_1', 'naics'])
        .assign(
            location_2 = df.location_1.astype(str).str[34:-2]
        )
    )

    df = df.assign(
        longitude = df.location_2.str.split(",", expand=True)[0].astype(float),
        latitude = df.location_2.str.split(",", expand=True)[1].astype(float),
        naics_sector = df.naics.str[:2].astype(str),
    ).dropna(subset=["longitude", "latitude"])

    # Merge in NAICS sector
    df2 = pd.merge(df, 
                   naics_table.assign(
                       naics_sector = naics_table.naics_sector.astype(str)
                   ), 
            how = 'inner', on = 'naics_sector', validate = 'm:1'
            )

    # Create geometry column
    gdf = gpd.GeoDataFrame(df2.dropna(subset=['longitude', 'latitude']), 
        geometry = gpd.points_from_xy(df2.longitude, df2.latitude),
                                      crs = "EPSG:4326"
    ).to_crs("EPSG:2229") # Change to CA State Plane

    # Import block groups
    block_group_file=(
        'https://raw.githubusercontent.com/CityofLosAngeles/civis-gcp-transition/{}/'
        'data/LACounty_Blockgroup.geojson'
    )
    block = gpd.read_file(block_group_file.format("active-business"))

    # Aggregate
    sjoin=gpd.sjoin(gdf, block, how='inner', op='intersects')
    
    sjoin = sjoin.assign(
        GEOID10 = sjoin.GEOID10.astype(str).apply(lambda x: '{0:0>12}'.format(x))
    )

    sjoin2=(sjoin.pivot_table(index='GEOID10', 
                    values='business_name',
                    columns=['naics_industry'], 
                    aggfunc=len)
        .reset_index()
        .fillna(0)
        .rename_axis(None, axis="columns")
    )
    
    # Merge geometry back in, since we lose the block group's polygon geometry when we aggregate
    sjoin3 = pd.merge(block, sjoin2, 
                      on = "GEOID10", how = "inner", validate = "1:1")
    
    return sjoin3


def top10(df):
    '''
    Find the top 10 predominant industries in entire county
    Exclude 2 categories
    Return a list (used to update feature layer item property)
    '''
    
    # Exclude these cols because we can't use idxmax on them
    exclude_cols = ['CTBG10', 'CT10', 'AreaSqMil', 'LABEL', 'FIP10', 'FIP10RV',
       'CDP_NAME', 'CITYNAME', 'COMMNAME', 'Shape_STAr', 'Shape_STLe',
       'geometry',]
    
    county_aggregate = (
        pd.DataFrame(df.drop(columns = exclude_cols)
                     .set_index("GEOID10")
                     .idxmax(axis=1))
        .reset_index()
        .rename(columns = {0: "predominant_industry"})
    )
    
    # Get a list, descending order
    predominant_industries = (county_aggregate.predominant_industry.value_counts()
                              .index
                              .to_list()
                             )
    
    # Exclude these categories, then grab top 10
    exclude_me = ['Professional, Scientific, and Technical Services', 
              'Other Services (except Public Administration)']
    for i in exclude_me:
        predominant_industries.remove(i)
    
    top10_industries = predominant_industries[0:10]
    
    return top10_industries



'''
ESRI stores the column names slightly differently (subject to 10 char limits)
Use dict to map and rename (key-value pair)
Key: dataframe's existing column name
Value: ESRI column name
'''
layer_rename_columns_dict = {
    'Accommodation and Food Services': 'Accommodation_and_Food_Services',
    'Administrative and Support and Waste Management and Remediation Services': 'Administrative_and_Support_and_',
    'Agriculture, Forestry, Fishing and Hunting': 'Agriculture__Forestry__Fishing_',
    'Arts, Entertainment, and Recreation': 'Arts__Entertainment__and_Recrea',
    'Construction': 'Construction',
    'Educational Services': 'Educational_Services',
    'Finance and Insurance': 'Finance_and_Insurance',
    'Health Care and Social Assistance': 'Health_Care_and_Social_Assistan',
    'Information': 'Information',
    'Manufacturing': 'Manufacturing',
    'Medical Marijuana Collective': 'Medical_Marijuana_Collective',
    'Mining': 'Mining',
    'Not Classified': 'Not_Classified',
    'Other Services (except Public Administration)': 'Other_Services__except_Public_A',
    'Professional, Scientific, and Technical Services': 'Professional__Scientific__and_T',
    'Real Estate Rental and Leasing': 'Real_Estate_Rental_and_Leasing',
    'Retail Trade': 'Retail_Trade',
    'Transportation and Warehousing': 'Transportation_and_Warehousing',
    'Utilities': 'Utilities',
    'Wholesale Trade': 'Wholesale_Trade'                                               
}



In [47]:
df=dataprep(abiz,naics_table)
top10_industries = top10(df)


In [113]:
block_group_file=(
    'https://raw.githubusercontent.com/CityofLosAngeles/civis-gcp-transition/{}/'
    'data/LACounty_Blockgroup.geojson'
)
block = gpd.read_file(block_group_file.format("active-business"))

In [115]:
block[block.GEOID10=="060379200000"]

,GEOID10,CTBG10,CT10,AreaSqMil,LABEL,FIP10,FIP10RV,CDP_NAME,CITYNAME,COMMNAME,Shape_STAr,Shape_STLe,geometry


In [49]:
#df.to_file(driver="GeoJSON", filename="./upload_me.geojson")

In [79]:
#upload_me = gpd.read_file("./upload_me.geojson").to_csv("./upload_me.csv", index=False)
gpd.read_file("./upload_me.geojson").rename(columns = {"geometry": "SHAPE"}).set_geometry("SHAPE").to_file(
driver="GeoJSON", filename="./upload_me.geojson")

In [58]:
tiffany_feature ="8a2f05cf6d454d38835bc24945d28588"

lahub_user = esri_credentials.tiffany_user
lahub_pass=esri_credentials.tiffany_pw


In [64]:
geohub_updates(df.assign(GEOID10 = df.GEOID10.astype(int)), 
               lahub_user, lahub_pass, tiffany_feature, 
               top10_industries, new_dict, OUTPUT_FILE)

NameError: name 'ActiveBusinseses_fset' is not defined

In [136]:
tiffany_feature ="8a2f05cf6d454d38835bc24945d28588"
preston_feature = "067a9242fbef4afeb1ca0744952e5724"
lahub_user = esri_credentials.preston_user
lahub_pass=esri_credentials.preston_pw


gis = GIS('https://lahub.maps.arcgis.com',  username=lahub_user, password=lahub_pass)
    
#flayer = gis.content.search(tiffany_feature)
flayer = gis.content.search(preston_feature)
ActiveBusinesses_item = flayer[0]
ActiveBusinesses_flayer = ActiveBusinesses_item.layers[0]
ActiveBusinesses_fset = ActiveBusinesses_flayer.query()

In [137]:
preston_df = ActiveBusinesses_fset.sdf

In [179]:
df2 = df.rename(columns = layer_rename_columns_dict)


merged_df = pd.merge(preston_df[["OBJECTID", "GEOID10", "SHAPE"]], 
        df2.drop(columns = ["geometry", 'Shape_STAr','Shape_STLe']), 
        on = "GEOID10", 
        how = "inner", 
        validate = "1:1")

integrify_me = list(layer_rename_columns_dict.values())

for c in integrify_me:
    merged_df[c] = merged_df[c].astype(int)

In [142]:
#preston_df.to_pickle("./preston_df.p")

In [76]:
ActiveBusinesses_fset = ActiveBusinesses_flayer.query()

In [77]:
ActiveBusinesses_fset

<FeatureSet> 6297 features

In [110]:
ActiveBusinesses_fset.sdf[ActiveBusinesses_fset.sdf.GEOID10==60379200000]

,GEOID10,CTBG10,CT10,AreaSqMil,LABEL,FIP10,FIP10RV,CDP_NAME,CITYNAME,COMMNAME,...,Professional__Scientific__and_T,Real_Estate_Rental_and_Leasing,Retail_Trade,Transportation_and_Warehousing,Utilities,Wholesale_Trade,ObjectId,Shape__Area,Shape__Length,SHAPE
472,60379200000,9200373,920037,0.0640,BG 3,NaN,99037,None,Unincorporated,LA County - Canyon Country,...,1,1,1,0,0,1,473,2.442149e+05,2667.806473,"{""rings"": [[[-13185366.5795035, 4084002.924334..."
473,60379200000,9201021,920102,8.5111,BG 1,NaN,99037,None,Unincorporated,LA County - Castaic/Lake Hughes,...,3,0,3,0,0,1,474,3.253123e+07,33640.462380,"{""rings"": [[[-13195575.135989, 4096671.3716552..."
544,60379200000,9203261,920326,0.5443,BG 1,74130.0,99037,Stevenson Ranch,Unincorporated,LA County - Stevenson Ranch,...,8,3,3,2,0,3,545,2.075330e+06,7054.463715,"{""rings"": [[[-13199106.5237193, 4080930.839724..."
545,60379200000,9203361,920336,0.2753,BG 1,69088.0,69088,Santa Clarita,Santa Clarita,City of Santa Clarita,...,1,0,0,0,0,0,546,1.049699e+06,4676.863481,"{""rings"": [[[-13195573.4650592, 4080785.155678..."
546,60379200000,9203362,920336,0.1342,BG 2,69088.0,69088,Santa Clarita,Santa Clarita,City of Santa Clarita,...,0,1,2,0,0,0,547,5.115935e+05,3233.016883,"{""rings"": [[[-13195399.8067539, 4081521.825048..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132,60379200000,9203344,920334,0.2352,BG 4,69088.0,69088,Santa Clarita,Santa Clarita,City of Santa Clarita,...,3,1,0,0,0,2,6133,8.967332e+05,4615.696634,"{""rings"": [[[-13196932.1197874, 4081641.617505..."
6178,60379200000,9200411,920041,0.6984,BG 1,NaN,99037,None,Unincorporated,LA County - Canyon Country,...,12,1,5,1,0,1,6179,2.665024e+06,7840.155723,"{""rings"": [[[-13182858.4397363, 4086028.952355..."
6179,60379200000,9200372,920037,0.0871,BG 2,NaN,99037,None,Unincorporated,LA County - Canyon Country,...,6,0,3,0,0,0,6180,3.322178e+05,3107.345321,"{""rings"": [[[-13185758.2015799, 4083959.476794..."
6180,60379200000,9200342,920034,1.1480,BG 2,NaN,99037,None,Unincorporated,LA County - Canyon Country,...,3,0,3,0,0,0,6181,4.382348e+06,11889.578616,"{""rings"": [[[-13182975.6594456, 4089209.473067..."


In [102]:
# overwrite some columns, but keep others
geographic_cols = ["GEOID10", 'CTBG10', 'CT10', 'AreaSqMil', 
                   'LABEL', 'FIP10', 'FIP10RV',
                   'CDP_NAME', 'CITYNAME', 'COMMNAME', 'Shape_STAr', 'Shape_STLe',
                   'ObjectId', 'Shape__Area', 'Shape__Length', 'SHAPE']

df2 = df.rename(columns = layer_rename_columns_dict)


keep_cols = ["GEOID10"] + list(layer_rename_columns_dict.values())
new_table = pd.merge(ActiveBusinesses_fset.sdf[geographic_cols],
                     df2[keep_cols].assign(GEOID10 = df2.GEOID10.astype(int)),
                     on = "GEOID10",
                     how = "left", 
                     validate = "1:1"
                    )

MergeError: Merge keys are not unique in left dataset; not a one-to-one merge

In [63]:
def geohub_updates(x,user,pas, feature_layer_id, 
        top10_industries, column_renaming_dict, OUTPUT_FILE):
    gis = GIS('https://lahub.maps.arcgis.com',  username=user, password=pas)
    # Try completely new layer first
    #feature_layer_id = '067a9242fbef4afeb1ca0744952e5724' # Preston's layer
    actbus=gis.content.search(feature_layer_id)
    ActiveBusinesses_item = actbus[0]
    ActiveBusinesses_flayer = ActiveBusinesses_item.layers[0]
    ActiveBusinesses_fset = ActiveBusinesses_flayer.query() #querying without any conditions returns all the features
    # Possibly not all the block groups are mapped
    # Select the block groups that are existing in the map (inner merge), and save those to CSV
    
    overlap_rows = x[x.assign(GEOID10 = x.GEOID10.astype(int)).GEOID10.isin(ActiveBusinesses_fset.sdf.GEOID10)]
    #overlap_rows = pd.merge(ActiveBusinesses_fset.sdf, 
    #                    x, 
    #                    how='inner',
    #                    on = 'GEOID10'
    #                    )


    # Rename columns to match
    overlap_rows = overlap_rows.rename(columns = column_renaming_dict)
    print(overlap_rows.columns)
    exclude_cols = [
        'CTBG10', 'CT10', 'AreaSqMil', 'LABEL', 'FIP10', 'FIP10RV',
        'CDP_NAME', 'CITYNAME', 'COMMNAME', 'Shape_STAr', 'Shape_STLe', 'geometry',]
    

    # Stage 2 files, one to check into GitHub (no geometry), one to use to back dashboard
    overlap_rows.to_csv("./upload_me.csv", index=False)
    overlap_rows.drop(columns = exclude_cols).to_csv(OUTPUT_FILE, index=False)
    
    # Overwrite table
    flayer = geohub.content.get(output_layer_name)
    flayer_collection = FeatureLayerCollection.fromitem(flayer)
    flayer_collection.manager.overwrite("./upload_me.csv")
    flayer_collection.manager.update_definition({"maxRecordCount": max_record_count})
    
    os.remove("./upload_me.csv")
    
    text = """
    This layer is aggregating 
    <a href="https://data.lacity.org/A-Prosperous-City/Listing-of-Active-Businesses/6rrh-rzua">
    Listing of Active Businesses Data</a> 
    that have geospatial information associated. 
    The top 10 most frequent industries in block groups are:
    {}
    """
    x = ', '.join([str(elem) for elem in top10_industries]) 
    item_props = {'title' : 'Active Businesses Data by Block Group', 'description':text.format(x)}
    # Use flayer.update or flayer_collection.update?
    flayer_collection.update(item_properties=item_props)
    print("updates made!")
